In [2]:
!pip install xarray[complete]

  Using cached xarray-2024.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached netCDF4-1.6.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached h5netcdf-1.3.0-py3-none-any.whl.metadata (13 kB)
  Using cached zarr-2.17.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached cftime-1.6.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.6 kB)
  Using cached pooch-1.8.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached nc_time_axis-1.4.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached hypothesis-6.99.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached pre_commit-3.6.2-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached pytest_cov-4.1.0-py3-none-any.whl.metadata (26 kB)
  Using cached pytest_env-1.1.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pytest_xdist-3.5.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached pytest_timeout-2.3.1-py3-none-any.whl.metadata

In [3]:
import xarray as xr
import s3fs
import boto3
import pandas as pd

In [6]:
s3 = s3fs.S3FileSystem()

In [4]:
s3ZarrPath = "s3://climate-action-datalake/zone=raw/source=agera5/variable=solarRadiationFlux.zarr/"
s3ZarrWindSpeed = "s3://climate-action-datalake/zone=raw/source=agera5/variable=windspeed.zarr/"
s3Humidity = "s3://climate-action-datalake/zone=raw/source=agera5/variable=relativehumidity.zarr/"
s3Precipitation = "s3://climate-action-datalake/zone=raw/source=chirps/variable=precipitation.zarr/"

In [7]:
storeWindSpeed = s3fs.S3Map(root=s3ZarrWindSpeed, s3=s3, check=False)
storeHumidity = s3fs.S3Map(root=s3Humidity, s3=s3, check=False)
storeSolarFlux = s3fs.S3Map(root=s3ZarrPath, s3=s3, check=False)
storePrecipitation = s3fs.S3Map(root=s3Precipitation, s3=s3, check=False)

In [8]:
precipitationZarr = xr.open_zarr(store=storePrecipitation, consolidated=True)
precipitationZarr

<xarray.Dataset> Size: 898GB
Dimensions:        (band: 1, lat: 2000, lon: 7200, time: 15594)
Coordinates:
  * band           (band) int64 8B 0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time           (time) datetime64[ns] 125kB 1981-01-01 ... 2023-08-10
Data variables:
    precipitation  (time, band, lat, lon) float32 898GB dask.array<chunksize=(1, 1, 2000, 7200), meta=np.ndarray>

In [9]:
precipitationZarr.sel(time='2023-08-10')

<xarray.Dataset> Size: 115MB
Dimensions:        (band: 1, lat: 2000, lon: 7200, time: 2)
Coordinates:
  * band           (band) int64 8B 0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time           (time) datetime64[ns] 16B 2023-08-10 2023-08-10
Data variables:
    precipitation  (time, band, lat, lon) float32 115MB dask.array<chunksize=(1, 1, 2000, 7200), meta=np.ndarray>

In [12]:
precipitationZarr.drop_duplicates(dim="time")

<xarray.Dataset> Size: 898GB
Dimensions:        (band: 1, lat: 2000, lon: 7200, time: 15583)
Coordinates:
  * band           (band) int64 8B 0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time           (time) datetime64[ns] 125kB 1981-01-01 ... 2023-08-31
Data variables:
    precipitation  (time, band, lat, lon) float32 898GB dask.array<chunksize=(1, 1, 2000, 7200), meta=np.ndarray>

In [13]:
precipitationZarr.drop_duplicates(dim="time").to_zarr("zarr/precipitation.zarr")

In [14]:
print("hola")

hola


In [15]:
a = xr.open_zarr("zarr/precipitation.zarr")

In [17]:
a.sel(time='2023-08-10')

<xarray.Dataset> Size: 58MB
Dimensions:        (band: 1, lat: 2000, lon: 7200)
Coordinates:
  * band           (band) int64 8B 0
  * lat            (lat) float64 16kB 49.97 49.92 49.87 ... -49.88 -49.93 -49.98
  * lon            (lon) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    time           datetime64[ns] 8B 2023-08-10
Data variables:
    precipitation  (band, lat, lon) float32 58MB dask.array<chunksize=(1, 2000, 7200), meta=np.ndarray>